In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
boxscore = pd.read_csv('NBA_2023_Halftime_Boxscore.csv')
game_ids = pd.read_csv('game_id.csv')

In [3]:
boxscore.head()

,GAME_ID,home_team,away_team,hs_minutes,hs_FGM,hs_FGA,hs_3PM,hs_3PA,hs_FTM,hs_FTA,...,at_3PA,at_FTM,at_FTA,at_OREB,at_DREB,at_AST,at_STL,at_BLK,at_TO,at_FOUL
0,22301195,NOP,LAL,65:32:00,11,26,6,15,6,6,...,14,3,3,4,17,23,7,1,6,7
1,22301192,PHI,BKN,76:20:00,15,32,5,12,5,6,...,17,3,4,2,20,11,1,4,4,6
2,22301194,MIN,PHX,84:34:00,12,17,5,7,7,9,...,16,5,6,6,5,16,10,2,6,11
3,22301191,ORL,MIL,87:59:00,12,31,3,13,8,9,...,16,12,13,5,18,7,6,2,7,7
4,22301187,CLE,CHA,76:51:00,12,24,4,7,4,4,...,25,2,3,6,20,19,4,3,5,5


In [4]:
game_ids.sample(5)

,Unnamed: 0,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,...,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,SEASON_TYPE,SEASON_YEAR
2454,3508,22023,1610612738,BOS,Boston Celtics,22300065,2023-10-25,BOS @ NYK,W,241,...,39,46,18,6,11,13,22,4.0,2,2023
757,1164,22023,1610612759,SAS,San Antonio Spurs,22300825,2024-02-25,SAS @ UTA,L,240,...,36,46,34,8,11,20,17,-19.0,2,2023
1964,3018,22023,1610612764,WAS,Washington Wizards,22300253,2023-11-27,WAS @ DET,W,240,...,40,46,29,8,6,15,19,19.0,2,2023
365,479,22023,1610612743,DEN,Denver Nuggets,22301026,2024-03-23,DEN @ POR,W,240,...,30,49,25,8,9,13,17,3.0,2,2023
1713,2767,22023,1610612738,BOS,Boston Celtics,22300344,2023-12-17,BOS vs. ORL,W,239,...,45,55,18,4,10,14,19,17.0,2,2023


### Cleaning *game_ids*

In [5]:
game_ids['home_away'] = game_ids.apply(lambda x: 'away' if '@' in x['MATCHUP'] else 'home', axis=1)

In [6]:
#### Create home_away column
game_ids['home_away'] = game_ids.apply(lambda x: 'away' if '@' in x['MATCHUP'] else 'home', axis=1)

#### Keep only necessary columns
game_ids = game_ids[['TEAM_ABBREVIATION', 'GAME_ID', 'PTS', 'home_away']]

### Cleaning *boxsocre*

In [7]:
boxscore.sample(5)

,GAME_ID,home_team,away_team,hs_minutes,hs_FGM,hs_FGA,hs_3PM,hs_3PA,hs_FTM,hs_FTA,...,at_3PA,at_FTM,at_FTA,at_OREB,at_DREB,at_AST,at_STL,at_BLK,at_TO,at_FOUL
951,22300726,CHA,TOR,73:24:00,14,20,6,10,6,9,...,18,10,11,2,15,13,1,1,4,8
1825,22301223,SAS,CHI,76:00:00,14,29,7,16,7,8,...,15,8,12,12,14,13,4,0,5,10
57,22301173,BOS,CHA,82:29:00,21,39,6,19,0,0,...,16,3,4,1,11,12,1,3,11,9
390,22301008,DAL,UTA,72:36:00,15,30,4,12,5,11,...,15,3,3,3,18,14,1,0,10,12
653,22300877,SAS,IND,81:29:00,16,34,4,10,2,2,...,15,5,7,5,19,9,5,2,8,5


In [8]:
boxscore.columns

Index(['GAME_ID', 'home_team', 'away_team', 'hs_minutes', 'hs_FGM', 'hs_FGA',
       'hs_3PM', 'hs_3PA', 'hs_FTM', 'hs_FTA', 'hs_OREB', 'hs_DREB', 'hs_AST',
       'hs_STL', 'hs_BLK', 'hs_TO', 'hs_FOUL', 'ht_minutes', 'ht_FGM',
       'ht_FGA', 'ht_3PM', 'ht_3PA', 'ht_FTM', 'ht_FTA', 'ht_OREB', 'ht_DREB',
       'ht_AST', 'ht_STL', 'ht_BLK', 'ht_TO', 'ht_FOUL', 'as_minutes',
       'as_FGM', 'as_FGA', 'as_3PM', 'as_3PA', 'as_FTM', 'as_FTA', 'as_OREB',
       'as_DREB', 'as_AST', 'as_STL', 'as_BLK', 'as_TO', 'as_FOUL',
       'at_minutes', 'at_FGM', 'at_FGA', 'at_3PM', 'at_3PA', 'at_FTM',
       'at_FTA', 'at_OREB', 'at_DREB', 'at_AST', 'at_STL', 'at_BLK', 'at_TO',
       'at_FOUL'],
      dtype='object')

In [9]:
# Calculate home team bench stats
boxscore['hb_FGM'] = boxscore['ht_FGM'] - boxscore['hs_FGM']
boxscore['hb_FGA'] = boxscore['ht_FGA'] - boxscore['hs_FGA']
boxscore['hb_3PM'] = boxscore['ht_3PM'] - boxscore['hs_3PM']
boxscore['hb_3PA'] = boxscore['ht_3PA'] - boxscore['hs_3PA']
boxscore['hb_FTM'] = boxscore['ht_FTM'] - boxscore['hs_FTM']
boxscore['hb_FTA'] = boxscore['ht_FTA'] - boxscore['hs_FTA']
boxscore['hb_OREB'] = boxscore['ht_OREB'] - boxscore['hs_OREB']
boxscore['hb_DREB'] = boxscore['ht_DREB'] - boxscore['hs_DREB']
boxscore['hb_AST'] = boxscore['ht_AST'] - boxscore['hs_AST']
boxscore['hb_STL'] = boxscore['ht_STL'] - boxscore['hs_STL']
boxscore['hb_BLK'] = boxscore['ht_BLK'] - boxscore['hs_BLK']
boxscore['hb_TO'] = boxscore['ht_TO'] - boxscore['hs_TO']
boxscore['hb_FOUL'] = boxscore['ht_FOUL'] - boxscore['hs_FOUL']

In [10]:
# Calculate away team bench stats
boxscore['ab_FGM'] = boxscore['at_FGM'] - boxscore['as_FGM']
boxscore['ab_FGA'] = boxscore['at_FGA'] - boxscore['as_FGA']
boxscore['ab_3PM'] = boxscore['at_3PM'] - boxscore['as_3PM']
boxscore['ab_3PA'] = boxscore['at_3PA'] - boxscore['as_3PA']
boxscore['ab_FTM'] = boxscore['at_FTM'] - boxscore['as_FTM']
boxscore['ab_FTA'] = boxscore['at_FTA'] - boxscore['as_FTA']
boxscore['ab_OREB'] = boxscore['at_OREB'] - boxscore['as_OREB']
boxscore['ab_DREB'] = boxscore['at_DREB'] - boxscore['as_DREB']
boxscore['ab_AST'] = boxscore['at_AST'] - boxscore['as_AST']
boxscore['ab_STL'] = boxscore['at_STL'] - boxscore['as_STL']
boxscore['ab_BLK'] = boxscore['at_BLK'] - boxscore['as_BLK']
boxscore['ab_TO'] = boxscore['at_TO'] - boxscore['as_TO']
boxscore['ab_FOUL'] = boxscore['at_FOUL'] - boxscore['as_FOUL']

In [11]:
# Drop team stats for both home and away
boxscore.drop(list(boxscore.filter(regex='t_')), axis=1, inplace=True)

### Merge *boxscore* and *game_ids* to create *final_df*

In [12]:
# Sort both dataframes by GAME_ID and concatenate side by side
boxscore = boxscore.sort_values(['GAME_ID']).reset_index(drop=True)
game_ids = game_ids.sort_values(['GAME_ID', 'home_away']).reset_index(drop=True)
combined_df = pd.concat([boxscore, game_ids], axis=1)
combined_df.head()

,GAME_ID,home_team,away_team,hs_minutes,hs_FGM,hs_FGA,hs_3PM,hs_3PA,hs_FTM,hs_FTA,...,ab_DREB,ab_AST,ab_STL,ab_BLK,ab_TO,ab_FOUL,TEAM_ABBREVIATION,GAME_ID,PTS,home_away
0,22300001,IND,CLE,76:53:00,18,33,6,11,6,8,...,6,4,2,1,1,4,CLE,22300001,116,away
1,22300001,IND,CLE,76:53:00,18,33,6,11,6,8,...,6,4,2,1,1,4,IND,22300001,121,home
2,22300002,MIL,NYK,74:21:00,12,25,5,8,8,10,...,5,1,0,0,3,1,NYK,22300002,105,away
3,22300002,MIL,NYK,74:21:00,12,25,5,8,8,10,...,5,1,0,0,3,1,MIL,22300002,110,home
4,22300003,MIA,WAS,73:25:00,13,25,4,7,6,6,...,4,7,1,0,3,0,WAS,22300003,114,away


In [13]:
# Split *combined_df* to home_df and away_df
home_df = combined_df[combined_df['home_away'] == 'home'].drop(list(combined_df.filter(regex='as_|ab_|TEAM_ABBREVIATION')), axis=1)
away_df = combined_df[combined_df['home_away'] == 'away'].drop(list(combined_df.filter(regex='hs_|hb_|TEAM_ABBREVIATION')), axis=1)

In [14]:
# Rename columns
home_df = home_df.rename({'home_team':'team', 'away_team':'against'}, axis=1).rename(columns=lambda x: re.sub(r'h(s|b)_', r'\1_', x)).reset_index(drop=True)
away_df = away_df.rename({'away_team':'team', 'home_team':'against'}, axis=1).rename(columns=lambda x: re.sub(r'a(s|b)_', r'\1_', x)).reset_index(drop=True)
# Remove duplicate column 'GAME_ID'
home_df = home_df.T.drop_duplicates().T
away_df = away_df.T.drop_duplicates().T

In [15]:
# Combine home_df and away_df
final_df = pd.concat([home_df, away_df],ignore_index=True, axis=0)

### Clean *final_df*

In [16]:
# Total score at halftime
final_df['PTS_ht'] = final_df.apply(lambda x: (x['s_FGM']+x['b_FGM'] - (x['s_3PM']+x['b_3PM']))*2 + (x['s_3PM']+x['b_3PM'])*3 + (x['s_FTM']+x['b_FTM'])*1 ,axis=1)

In [17]:
# Sort by GAME_ID
final_df.sort_values('GAME_ID', inplace=True)

In [18]:
final_df.head()

,GAME_ID,team,against,s_minutes,s_FGM,s_FGA,s_3PM,s_3PA,s_FTM,s_FTA,...,b_OREB,b_DREB,b_AST,b_STL,b_BLK,b_TO,b_FOUL,PTS,home_away,PTS_ht
0,22300001,IND,CLE,76:53:00,18,33,6,11,6,8,...,0,7,5,1,3,4,3,121,home,70
1230,22300001,CLE,IND,70:40:00,16,30,2,9,4,5,...,0,6,4,2,1,1,4,116,away,53
1231,22300002,NYK,MIL,87:47:00,13,39,5,15,7,12,...,2,5,1,0,0,3,1,105,away,46
1,22300002,MIL,NYK,74:21:00,12,25,5,8,8,10,...,0,6,3,3,0,0,3,110,home,56
2,22300003,MIA,WAS,73:25:00,13,25,4,7,6,6,...,3,3,7,2,1,2,1,121,home,60


In [19]:
# Convert s_minutes from string to number of seconds (b_minutes not needed as it's directly correlated with s_minutes)
final_df['s_minutes'] = final_df['s_minutes'].apply(lambda x: int(x.split(':')[0])+int(x.split(':')[1])/60)

In [20]:
# rename PTS to PTS_final for clarification
final_df.rename({'PTS':'PTS_final'}, axis=1, inplace=True)

In [21]:
final_df.head()

,GAME_ID,team,against,s_minutes,s_FGM,s_FGA,s_3PM,s_3PA,s_FTM,s_FTA,...,b_OREB,b_DREB,b_AST,b_STL,b_BLK,b_TO,b_FOUL,PTS_final,home_away,PTS_ht
0,22300001,IND,CLE,76.883333,18,33,6,11,6,8,...,0,7,5,1,3,4,3,121,home,70
1230,22300001,CLE,IND,70.666667,16,30,2,9,4,5,...,0,6,4,2,1,1,4,116,away,53
1231,22300002,NYK,MIL,87.783333,13,39,5,15,7,12,...,2,5,1,0,0,3,1,105,away,46
1,22300002,MIL,NYK,74.350000,12,25,5,8,8,10,...,0,6,3,3,0,0,3,110,home,56
2,22300003,MIA,WAS,73.416667,13,25,4,7,6,6,...,3,3,7,2,1,2,1,121,home,60


In [22]:
# Create column for opponent point at halftime
final_df['GAME_ID_agst'] = final_df.apply(lambda x:str(x['GAME_ID']) + x['against'], axis=1)
final_df['GAME_ID_team'] = final_df.apply(lambda x:str(x['GAME_ID']) + x['team'], axis=1)

In [23]:
# Create dataframe for opponent point at halftime only, use this to merge
opp_pts = final_df[['GAME_ID_agst', 'PTS_ht']]
# Merge final_df and opp_pts
final_df = final_df.merge(opp_pts, left_on='GAME_ID_team', right_on='GAME_ID_agst')
final_df.head()

,GAME_ID,team,against,s_minutes,s_FGM,s_FGA,s_3PM,s_3PA,s_FTM,s_FTA,...,b_BLK,b_TO,b_FOUL,PTS_final,home_away,PTS_ht_x,GAME_ID_agst_x,GAME_ID_team,GAME_ID_agst_y,PTS_ht_y
0,22300001,IND,CLE,76.883333,18,33,6,11,6,8,...,3,4,3,121,home,70,22300001CLE,22300001IND,22300001IND,53
1,22300001,CLE,IND,70.666667,16,30,2,9,4,5,...,1,1,4,116,away,53,22300001IND,22300001CLE,22300001CLE,70
2,22300002,NYK,MIL,87.783333,13,39,5,15,7,12,...,0,3,1,105,away,46,22300002MIL,22300002NYK,22300002NYK,56
3,22300002,MIL,NYK,74.350000,12,25,5,8,8,10,...,0,0,3,110,home,56,22300002NYK,22300002MIL,22300002MIL,46
4,22300003,MIA,WAS,73.416667,13,25,4,7,6,6,...,1,2,1,121,home,60,22300003WAS,22300003MIA,22300003MIA,54


In [24]:
# Drop uncessary columns
final_df.drop(columns=['GAME_ID_team', 'GAME_ID_agst_x', 'GAME_ID_agst_y'], inplace=True)
# Rename columns
final_df.rename(columns={'PTS_ht_y': 'PTS_ht_opp', 'PTS_ht_x': 'PTS_ht'}, inplace=True)
final_df.head()

,GAME_ID,team,against,s_minutes,s_FGM,s_FGA,s_3PM,s_3PA,s_FTM,s_FTA,...,b_DREB,b_AST,b_STL,b_BLK,b_TO,b_FOUL,PTS_final,home_away,PTS_ht,PTS_ht_opp
0,22300001,IND,CLE,76.883333,18,33,6,11,6,8,...,7,5,1,3,4,3,121,home,70,53
1,22300001,CLE,IND,70.666667,16,30,2,9,4,5,...,6,4,2,1,1,4,116,away,53,70
2,22300002,NYK,MIL,87.783333,13,39,5,15,7,12,...,5,1,0,0,3,1,105,away,46,56
3,22300002,MIL,NYK,74.350000,12,25,5,8,8,10,...,6,3,3,0,0,3,110,home,56,46
4,22300003,MIA,WAS,73.416667,13,25,4,7,6,6,...,3,7,2,1,2,1,121,home,60,54


In [25]:
# Move PTS_final (response variable) to end
PTS_final = final_df.pop('PTS_final')
final_df.insert(len(final_df.columns), 'PTS_final', PTS_final)
final_df.head()

,GAME_ID,team,against,s_minutes,s_FGM,s_FGA,s_3PM,s_3PA,s_FTM,s_FTA,...,b_DREB,b_AST,b_STL,b_BLK,b_TO,b_FOUL,home_away,PTS_ht,PTS_ht_opp,PTS_final
0,22300001,IND,CLE,76.883333,18,33,6,11,6,8,...,7,5,1,3,4,3,home,70,53,121
1,22300001,CLE,IND,70.666667,16,30,2,9,4,5,...,6,4,2,1,1,4,away,53,70,116
2,22300002,NYK,MIL,87.783333,13,39,5,15,7,12,...,5,1,0,0,3,1,away,46,56,105
3,22300002,MIL,NYK,74.350000,12,25,5,8,8,10,...,6,3,3,0,0,3,home,56,46,110
4,22300003,MIA,WAS,73.416667,13,25,4,7,6,6,...,3,7,2,1,2,1,home,60,54,121


In [27]:
# Export data to 'NBA_2023_halftime_boxscore_data_for_analysis.csv'
final_df.to_csv('NBA_2023_halftime_boxscore_data_for_analysis.csv')